In [1]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib

In [2]:
def graddesc(A, b, x, tol):
    """
    Compute gradient descent for function norm(Ax-b)
    """
    # Compute the gradient
    r = np.dot(A.transpose(),b-np.dot(A,x))
    # Start with an empty array
    xout = [x]
    while la.norm(r,2) > tol:
        # If the gradient is bigger than the tolerance
        Ar = np.dot(A,r)
        alpha = np.dot(r,r)/np.dot(Ar,Ar)
        x = x + alpha*r
        xout.append(x)
        r = r-alpha*np.dot(A.transpose(),Ar)
    return np.array(xout).transpose()

In [3]:
class GraddescBuilder:
    def __init__(self, line, cont, A, b, x, tol):
        self.line = line
        self.cont = cont
        self.xs = line.get_xdata()
        self.ys = line.get_ydata()
        self.cid = line.figure.canvas.mpl_connect('button_press_event', self)

    def __call__(self, event):
        #print('click', event)
        if event.inaxes!=self.line.axes: return
        self.xs = event.xdata
        self.ys = event.ydata
        
        xout = graddesc(A, b, np.array([self.xs, self.ys]), tol)
        
        fvals = []
        for i in range(7):
            fvals.append(f(xout[:,i]))
        
        self.line.set_data(xout[0,:], xout[1,:])
        self.cont.levels = np.array(list(reversed(fvals)))
        plt.draw()
        #self.line.figure.canvas.draw()


In [4]:
% matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111)

A = np.array([[2,0], [1,3], [0,1]])
b = np.array([1, -1, 0])
tol = 1e-4
x = np.zeros(2)

# Define the function we aim to minimize
def f(x):
    return np.dot(np.dot(A,x)-b,np.dot(A,x)-b)

# Create a mesh grid 
xx = np.linspace(-0.1,0.9,100)
yy = np.linspace(-0.1,-0.8,100)
X, Y = np.meshgrid(xx, yy)

Z = np.zeros(X.shape)
for i in range(Z.shape[0]):
    for j in range(Z.shape[1]):
        Z[i,j] = f(np.array([X[i,j], Y[i,j]]))

# Get a nice monotone colormap
cmap = plt.cm.get_cmap("coolwarm")

# Plot the contours and the trajectory
cont = ax.contour(X, Y, Z, cmap = cmap)
#plt.plot(traj[0,:], traj[1,:], 'o-k')
plt.show()

#def onclick(event):
#    ax.plot([event.xdata], [event.ydata], 'o', size=3)
    #print event.xdata, event.ydata

#cid = fig.canvas.mpl_connect('button_press_event', onclick)

#fig = plt.figure()
#ax = fig.add_subplot(111)
#ax.set_title('click to build line segments')
line, = ax.plot([0], [0], color='black', linewidth=3)  # empty line
linebuilder = GraddescBuilder(line, cont, A, b, x, tol)

plt.show()